In [153]:
from datetime import datetime
import itertools
import pandas as pd
import numpy as np
import requests
import json
from tqdm.auto import tqdm

In [4]:
solar_home_data = pd.read_csv("data/2010-2011 Solar home electricity data.csv")

In [8]:
np.unique([(solar_home_data.Customer == i).sum() for i in solar_home_data.Customer.unique()])

array([ 730, 1095])

Each customer has either two or three types of recordings for each day of the year. The recordings correspond to the consumption categories.

In [14]:
solar_home_data

,Customer,Generator Capacity,Postcode,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,3.78,2076,GC,1-Jul-10,0.303,0.471,0.083,0.121,0.361,...,0.495,0.540,0.406,0.543,0.495,0.216,0.378,0.128,0.078,0.125
1,1,3.78,2076,CL,1-Jul-10,1.250,1.244,1.256,0.744,0.019,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.075
2,1,3.78,2076,GG,1-Jul-10,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,3.78,2076,GC,2-Jul-10,0.116,0.346,0.122,0.079,0.120,...,0.262,1.100,1.012,0.817,0.526,0.335,0.402,0.142,0.120,0.111
4,1,3.78,2076,CL,2-Jul-10,1.238,1.238,1.256,1.250,0.169,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269730,300,3.36,2086,CL,29-Jun-11,2.136,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.490
269731,300,3.36,2086,GG,29-Jun-11,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
269732,300,3.36,2086,GC,30-Jun-11,1.306,0.851,0.113,0.151,0.104,...,0.948,0.917,0.872,0.832,0.863,0.789,0.280,0.224,0.259,0.774
269733,300,3.36,2086,CL,30-Jun-11,0.904,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.528


In [7]:
solar_home_data.Postcode.unique()

array([2076, 2259, 2263, 2220, 2261, 2230, 2126, 2262, 2026, 2218, 2265,
       2208, 2229, 2085, 2250, 2283, 2305, 2285, 2300, 2093, 2025, 2041,
       2303, 2082, 2284, 2137, 2048, 2282, 2205, 2224, 2021, 2290, 2010,
       2135, 2251, 2162, 2227, 2295, 2190, 2029, 2204, 2031, 2039, 2087,
       2074, 2330, 2291, 2119, 2099, 2066, 2100, 2081, 2320, 2134, 2280,
       2289, 2294, 2159, 2008, 2209, 2203, 2278, 2079, 2228, 2044, 2046,
       2111, 2077, 2018, 2325, 2324, 2196, 2231, 2120, 2260, 2292, 2293,
       2086, 2207, 2096, 2047, 2233, 2034, 2092, 2264, 2037, 2296, 2286,
       2200, 2219, 2315, 2211, 2154, 2088, 2198, 2326, 2084, 2321, 2281,
       2216])

In [5]:
postcode_data = pd.read_csv("data/AU.csv")

In [6]:
postcode_data

,country code,postal code,place name,admin name1,admin code1,admin name2,admin code2,admin name3,admin code3,latitude,longitude,accuracy
0,AU,200,Australian National University,Australian Capital Territory,ACT,CANBERRA,NaN,NaN,NaN,-35.2777,149.1189,1.0
1,AU,221,Barton,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.3049,149.1412,4.0
2,AU,2540,Jervis Bay,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.1499,150.6969,4.0
3,AU,2540,Wreck Bay,Australian Capital Territory,ACT,NaN,NaN,NaN,NaN,-35.1648,150.6908,4.0
4,AU,2540,Hmas Creswell,Australian Capital Territory,ACT,NEW CNTRY WEST,NaN,NaN,NaN,-35.0280,150.5501,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16868,AU,6989,Maddington,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0500,115.9833,4.0
16869,AU,6990,Gosnells,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.0810,116.0054,4.0
16870,AU,6991,Kelmscott,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1243,116.0259,4.0
16871,AU,6992,Armadale,Western Australia,WA,TANGNEY,NaN,NaN,NaN,-32.1461,116.0093,4.0


In [28]:
postcode_data = postcode_data.drop(postcode_data[postcode_data['postal code'].duplicated()].index)

In [124]:
def add_coord(datarow):
    postcode_row = postcode_data[datarow.Postcode == postcode_data['postal code']]
    coords = pd.Series({"longitude": postcode_row.longitude.item(), "latitude": postcode_row.latitude.item()})
    return pd.concat((datarow, coords))


solar_home_data = solar_home_data.apply(add_coord, axis=1)

Convert to net energy, make the dataset hourly, and put it in long form

In [106]:
new_data = []
for customer in tqdm(solar_home_data.Customer.unique()):
    for date in solar_home_data.date.unique():
        rows = solar_home_data[(solar_home_data.Customer == customer) & (solar_home_data.date == date)]
        consumption_rows = rows[(rows['Consumption Category'] == "GC") | (rows['Consumption Category'] == "CL")]
        generation_row = rows[rows['Consumption Category'] == "GG"]
        time_columns = rows.columns[5:]
        generation_row.loc[:, time_columns] = consumption_rows[time_columns].sum() - generation_row[time_columns]
        new_row = generation_row.drop(columns=["Consumption Category"])
        new_data.append(new_row)
solar_home_data = pd.concat(new_data)

100%|██████████| 10/10 [00:43<00:00,  4.32s/it]


,Customer,Generator Capacity,Postcode,date,0:30,1:00,1:30,2:00,2:30,3:00,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
2,1,3.78,2076,1-Jul-10,1.553,1.715,1.339,0.865,0.380,1.329,...,0.495,0.540,0.406,0.543,0.495,0.216,0.378,0.128,0.078,1.200
5,1,3.78,2076,2-Jul-10,1.354,1.584,1.378,1.329,0.289,0.701,...,0.262,1.100,1.012,0.817,0.526,0.335,0.402,0.142,0.120,1.199
8,1,3.78,2076,3-Jul-10,1.980,2.633,2.042,1.887,0.729,1.674,...,0.441,0.387,0.274,0.782,0.519,0.225,0.123,0.157,0.390,1.205
11,1,3.78,2076,4-Jul-10,1.399,1.333,1.393,1.321,0.123,1.300,...,1.087,0.992,0.462,0.379,0.505,0.586,0.269,0.252,0.295,1.313
14,1,3.78,2076,5-Jul-10,1.545,1.725,1.645,1.316,0.122,1.203,...,0.904,0.945,0.385,0.295,1.034,0.169,0.122,0.135,0.079,1.453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10937,10,1.05,2262,26-Jun-11,2.636,0.221,0.178,0.856,1.802,1.159,...,0.852,0.796,0.786,2.546,1.400,1.279,1.011,1.016,3.382,3.332
10940,10,1.05,2262,27-Jun-11,3.333,3.369,3.483,3.433,2.998,0.300,...,1.493,2.267,1.919,1.127,1.006,1.412,1.004,0.946,3.456,3.381
10943,10,1.05,2262,28-Jun-11,3.395,3.388,3.468,3.411,2.468,0.828,...,1.506,1.556,1.530,1.146,0.441,0.217,0.206,0.236,3.164,3.401
10946,10,1.05,2262,29-Jun-11,2.928,2.715,1.817,0.183,0.182,0.173,...,1.136,1.056,0.986,0.985,0.929,0.958,0.918,0.998,2.921,2.643


In [140]:
# Get rid of half hour intervals
df = solar_home_data.copy()
time_columns = solar_home_data.columns[5:-2]
for half_hour, hour in zip(time_columns[::2], time_columns[1::2]):
    df[hour] += df[half_hour]
df = df.drop(columns=time_columns[::2])
df

,Customer,Generator Capacity,Postcode,Consumption Category,date,1:00,2:00,3:00,4:00,5:00,...,17:00,18:00,19:00,20:00,21:00,22:00,23:00,0:00,longitude,latitude
0,1,3.78,2076,GC,1-Jul-10,0.774,0.204,0.440,0.220,0.473,...,3.549,2.656,0.795,1.035,0.949,0.711,0.506,0.203,151.0974,-33.7232
1,1,3.78,2076,CL,1-Jul-10,2.494,2.000,1.269,0.069,0.869,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.075,151.0974,-33.7232
2,1,3.78,2076,GG,1-Jul-10,0.000,0.000,0.000,0.000,0.000,...,0.031,0.000,0.000,0.000,0.000,0.000,0.000,0.000,151.0974,-33.7232
3,1,3.78,2076,GC,2-Jul-10,0.462,0.201,0.227,0.463,0.202,...,0.680,2.373,0.960,1.362,1.829,0.861,0.544,0.231,151.0974,-33.7232
4,1,3.78,2076,CL,2-Jul-10,2.476,2.506,0.763,0.688,0.456,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.088,151.0974,-33.7232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269730,300,3.36,2086,CL,29-Jun-11,2.136,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.490,151.2232,-33.7482
269731,300,3.36,2086,GG,29-Jun-11,0.000,0.000,0.000,0.000,0.000,...,0.047,0.000,0.000,0.000,0.000,0.000,0.000,0.000,151.2232,-33.7482
269732,300,3.36,2086,GC,30-Jun-11,2.157,0.264,0.253,0.242,0.340,...,2.251,3.701,2.257,1.865,1.704,1.652,0.504,1.033,151.2232,-33.7482
269733,300,3.36,2086,CL,30-Jun-11,0.904,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.528,151.2232,-33.7482


In [141]:
id_columns = df.columns[:5].tolist() + df.columns[-2:].tolist()
time_columns = df.columns[5:-2]
df = pd.melt(df, id_vars=id_columns, value_vars=time_columns)
df

,Customer,Generator Capacity,Postcode,Consumption Category,date,longitude,latitude,variable,value
0,1,3.78,2076,GC,1-Jul-10,151.0974,-33.7232,1:00,0.774
1,1,3.78,2076,CL,1-Jul-10,151.0974,-33.7232,1:00,2.494
2,1,3.78,2076,GG,1-Jul-10,151.0974,-33.7232,1:00,0.000
3,1,3.78,2076,GC,2-Jul-10,151.0974,-33.7232,1:00,0.462
4,1,3.78,2076,CL,2-Jul-10,151.0974,-33.7232,1:00,2.476
...,...,...,...,...,...,...,...,...,...
6473635,300,3.36,2086,CL,29-Jun-11,151.2232,-33.7482,0:00,2.490
6473636,300,3.36,2086,GG,29-Jun-11,151.2232,-33.7482,0:00,0.000
6473637,300,3.36,2086,GC,30-Jun-11,151.2232,-33.7482,0:00,1.033
6473638,300,3.36,2086,CL,30-Jun-11,151.2232,-33.7482,0:00,2.528


In [144]:
df = df.rename(columns={"value": "Net Energy"})
df['date'] += " " + df['variable']
df.drop(columns='variable')
df['date'] = df['date'].map(lambda x: datetime.strptime(x, "%d-%b-%y %H:%M").isoformat(timespec="minutes"))
df

,Customer,Generator Capacity,Postcode,Consumption Category,date,longitude,latitude,variable,Net Energy
0,1,3.78,2076,GC,1-Jul-10,151.0974,-33.7232,1:00,0.774
1,1,3.78,2076,CL,1-Jul-10,151.0974,-33.7232,1:00,2.494
2,1,3.78,2076,GG,1-Jul-10,151.0974,-33.7232,1:00,0.000
3,1,3.78,2076,GC,2-Jul-10,151.0974,-33.7232,1:00,0.462
4,1,3.78,2076,CL,2-Jul-10,151.0974,-33.7232,1:00,2.476
...,...,...,...,...,...,...,...,...,...
6473635,300,3.36,2086,CL,29-Jun-11,151.2232,-33.7482,0:00,2.490
6473636,300,3.36,2086,GG,29-Jun-11,151.2232,-33.7482,0:00,0.000
6473637,300,3.36,2086,GC,30-Jun-11,151.2232,-33.7482,0:00,1.033
6473638,300,3.36,2086,CL,30-Jun-11,151.2232,-33.7482,0:00,2.528


 Add weather data

In [84]:
# Download weather data
for postcode in tqdm(solar_home_data.Postcode.unique()):
    postcode_row = postcode_data[postcode == postcode_data['postal code']]
    latitude, longitude = postcode_row.latitude.item(), postcode_row.longitude.item()
    r = requests.get(f"https://archive-api.open-meteo.com/v1/era5?latitude={latitude}&longitude={longitude}&start_date=2010-07-01&end_date=2011-06-30&hourly=temperature_2m,precipitation,cloudcover")
    if r.ok:
        with open(f"data/weather/{postcode}.json", 'w') as f:
            json.dump(r.json(), f)
    else:
        print(f"Postcode {postcode} weather data request failed")

100%|██████████| 100/100 [04:08<00:00,  2.49s/it]


In [196]:
df['Temperature'] = 0
df['Precipitation'] = 0
df['Cloud cover'] = 0
for postcode in df.Postcode.unique():
    with open(f"data/weather/{postcode}.json", 'r') as f:
        weather = json.load(f)
    for i, time in enumerate(weather['hourly']['time']):
        # print(f"{postcode=}, {time=}, {weather['hourly']['temperature_2m'][i]=}, {weather['hourly']['precipitation'][i]=}, {weather['hourly']['cloudcover'][i]=}")
        row_idx = df[(df.Postcode == postcode) & (df.date == time)].index
        df.loc[row_idx, 'Temperature'] = weather['hourly']['temperature_2m'][i]
        df.loc[row_idx, 'Precipitation'] = weather['hourly']['precipitation'][i]
        df.loc[row_idx, 'Cloud cover'] = weather['hourly']['cloudcover'][i]

postcode=2076, time='2010-07-01T00:00', weather['hourly']['temperature_2m'][i]=9.3, weather['hourly']['precipitation'][i]=0.0, weather['hourly']['cloudcover'][i]=7
postcode=2076, time='2010-07-01T01:00', weather['hourly']['temperature_2m'][i]=11.1, weather['hourly']['precipitation'][i]=0.0, weather['hourly']['cloudcover'][i]=2
postcode=2076, time='2010-07-01T02:00', weather['hourly']['temperature_2m'][i]=12.2, weather['hourly']['precipitation'][i]=0.0, weather['hourly']['cloudcover'][i]=2
postcode=2076, time='2010-07-01T03:00', weather['hourly']['temperature_2m'][i]=12.9, weather['hourly']['precipitation'][i]=0.0, weather['hourly']['cloudcover'][i]=0
postcode=2076, time='2010-07-01T04:00', weather['hourly']['temperature_2m'][i]=13.4, weather['hourly']['precipitation'][i]=0.1, weather['hourly']['cloudcover'][i]=22
postcode=2076, time='2010-07-01T05:00', weather['hourly']['temperature_2m'][i]=13.5, weather['hourly']['precipitation'][i]=0.0, weather['hourly']['cloudcover'][i]=48
postcode=

In [197]:
df

,Customer,Generator Capacity,Postcode,Consumption Category,date,longitude,latitude,variable,Net Energy,Temperature,Precipitation,Cloud cover
0,1,3.78,2076,GC,2010-07-01T01:00,151.0974,-33.7232,1:00,0.774,11.1,0.0,2
1,1,3.78,2076,CL,2010-07-01T01:00,151.0974,-33.7232,1:00,2.494,11.1,0.0,2
2,1,3.78,2076,GG,2010-07-01T01:00,151.0974,-33.7232,1:00,0.000,11.1,0.0,2
3,1,3.78,2076,GC,2010-07-02T01:00,151.0974,-33.7232,1:00,0.462,0.0,0.0,0
4,1,3.78,2076,CL,2010-07-02T01:00,151.0974,-33.7232,1:00,2.476,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6473635,300,3.36,2086,CL,2011-06-29T00:00,151.2232,-33.7482,0:00,2.490,0.0,0.0,0
6473636,300,3.36,2086,GG,2011-06-29T00:00,151.2232,-33.7482,0:00,0.000,0.0,0.0,0
6473637,300,3.36,2086,GC,2011-06-30T00:00,151.2232,-33.7482,0:00,1.033,0.0,0.0,0
6473638,300,3.36,2086,CL,2011-06-30T00:00,151.2232,-33.7482,0:00,2.528,0.0,0.0,0
